In [25]:
#@title Libraries, API keys and imports.
!pip install -qU langchain
!pip install -qU huggingface_hub
!pip install -qU openai
!pip install -qU google-cloud-aiplatform "shapely<2"
import os

# Generate a new key here: https://platform.openai.com/account/api-keys
os.environ['OPENAI_API_KEY'] = "your_key_here"

In [26]:
#@title Gcloud setup to use VertexAI.
!gcloud auth application-default login
!gcloud config set project stellar-works-391900

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=pZDP4j7MSStVuWC7JuBkFWvmYTnJYl&prompt=consent&access_type=offline&code_challenge=M1X37MlNa8woJ_hl5NUBzhv4379x2GBWpPggqejil9U&code_challenge_method=S256

Enter authorization code: 4/0AZEOvhVTF-Oj7xD3dbjqRMe7LHA203TA6LKCW0y4F0cW-WIGGwNXM6HQ94_LryW8LUDTwg

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "stellar-works-391900" wa

In [27]:
#@title 1. Instruction following.

from langchain.llms import OpenAI
from langchain.llms import VertexAI

llm = OpenAI(model="text-davinci-003", temperature=0.9)
# llm = VertexAI()
text = "Suggest a personalized workout routine for someone looking to improve heart health outdoors."
print(llm(text))



1. Start with a 10-minute warmup, such as walking, jogging, or cycling at a moderate pace.

2. Do 20 minutes of interval training by increasing your pace to a fast run for 1 minute, then slowing down for 2 minutes. Repeat this cycle 8 times.

3. Follow this with 10 minutes of total body stretching.

4. Jog or walk briskly for 15 minutes, followed by a 5-minute cool down.

5. On alternate days, try going for a 30-minute power walk or a 30-minute light jog.

6. Add 30 minutes of swimming or cycling for 2 or 3 days a week.

7. Finish with 10 minutes of stretching and 5 minutes of deep breathing.


In [28]:
#@title 2. LLMChain - prompt template with embedded inputs.

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["course"],
    template="What is a good name for a company that teaches {course}?",
)

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run("recommendation systems"))
# TODO: build a pydantic output parser to programmatically extract the answer.



RecSysTutor.


In [29]:
#@title 3. ConversationChain with memory.
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

# conversation
conversation.predict(input="Introduce yourself.")
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with teaching LLMs?")

print(conversation)



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Introduce yourself.
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Introduce yourself.
AI:  Hi there! I'm a chatbot created to help make your day a little easier. My name is Botty and I'm here to help answer any questions or lend an ear if you need to chat. What can I do for you today?
Human: What can you do?
AI:

> Finished chain.


> Entering new  chain...
Prompt after formatting:
The followi

In [30]:
#@title 4. LLMChain with prompt template to generate recommendations.

# Clear instructions.
# Specificity.
# Open-ended creativity.
# Focus on the task.
prompt = PromptTemplate(
    input_variables=["theme", "year"],
    template="You must help me come up with a song title. What's a good song title for a song about {theme} in the year {year}?",
)

# Create LLMChain
chain = LLMChain(llm=llm, prompt=prompt)
response = chain.run({
    "theme": "interstellar travel",
    "year": "3030"
})

print("Theme: interstellar travel")
print("Year: 3030")
print("AI-generated song title:", response)

Theme: interstellar travel
Year: 3030
AI-generated song title: 

"Interstellar Odyssey: 3030"


In [31]:
#@title 5. Few-shot prompting.
from langchain import FewShotPromptTemplate

examples = [
    {"animal": "lion", "habitat": "savanna"},
    {"animal": "polar bear", "habitat": "Arctic ice"},
    {"animal": "elephant", "habitat": "African grasslands"}
]

example_template = """
Animal: {animal}
Habitat: {habitat}
"""

example_prompt = PromptTemplate(
    input_variables=["animal", "habitat"],
    template=example_template
)

dynamic_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Identify the habitat of the given animal",
    suffix="Animal: {input}\nHabitat:",
    input_variables=["input"],
    example_separator="\n\n",
)

# Create the LLMChain for the dynamic_prompt
chain = LLMChain(llm=llm, prompt=dynamic_prompt)

# Run the LLMChain with input_data
input_data = {"input": "tiger"}
response = chain.run(input_data)

print(response)

 tropical and subtropical forests


In [32]:
#@title 6. Extracting knowledge triplets.
# Source: https://github.com/hwchase17/langchain/blob/master/langchain/indexes/prompts/knowledge_triplet_extraction.py

from langchain.graphs.networkx_graph import KG_TRIPLE_DELIMITER

# Prompt template for knowledge triple extraction
_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE = (
    "You are a networked intelligence helping a human track knowledge triples"
    " about all relevant people, things, concepts, etc. and integrating"
    " them with your knowledge stored within your weights"
    " as well as that stored in a knowledge graph."
    " Extract all of the knowledge triples from the text."
    " A knowledge triple is a clause that contains a subject, a predicate,"
    " and an object. The subject is the entity being described,"
    " the predicate is the property of the subject that is being"
    " described, and the object is the value of the property.\n\n"
    "EXAMPLE\n"
    "It's a state in the US. It's also the number 1 producer of gold in the US.\n\n"
    f"Output: (Nevada, is a, state){KG_TRIPLE_DELIMITER}(Nevada, is in, US)"
    f"{KG_TRIPLE_DELIMITER}(Nevada, is the number 1 producer of, gold)\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "I'm going to the store.\n\n"
    "Output: NONE\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "Oh huh. I know Descartes likes to drive antique scooters and play the mandolin.\n"
    f"Output: (Descartes, likes to drive, antique scooters){KG_TRIPLE_DELIMITER}(Descartes, plays, mandolin)\n"
    "END OF EXAMPLE\n\n"
    "EXAMPLE\n"
    "{text}"
    "Output:"
)

KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT = PromptTemplate(
    input_variables=["text"],
    template=_DEFAULT_KNOWLEDGE_TRIPLE_EXTRACTION_TEMPLATE,
)

# Create an LLMChain using the knowledge triple extraction prompt
chain = LLMChain(llm=llm, prompt=KNOWLEDGE_TRIPLE_EXTRACTION_PROMPT)

# Run the chain with the specified text
text = "The city of Paris is the capital and most populous city of France. The Eiffel Tower is a famous landmark in Paris."
triples = chain.run(text)

print(triples)

 (Paris, is the capital of, France)<|>(Paris, is the most populous city of, France)<|>(Eiffel Tower, is a, landmark)<|>(Eiffel Tower, is in, Paris)
